In [1]:
import numpy as np

### Specific constants of the System
The main constants are defined first

In [23]:
BYTE_SIZE = 64
RAM_SIZE = 4000
LESS = -1
EQUAL = 0
GREATER = 1

### Word with an arbitrary size
Here we specify the MIX machine Word structure, with an arbitrary number of Bytes and a sign. We implement it as a numpy array of Size+1 number of integers. 

The first (zeros) integer is the Sign 

`Word[0]=sign`

In [17]:
class GenericWord:
    def __init__(self, size) -> None:
        self.__value__ = np.zeros((size+1), dtype=np.int64)
        self.__value__[0] = 1 # initialize the sign as "+"
        self.__size__ = size

    @property
    def size(self):
        return self.__size__

    @property
    def sign(self):
        if self.__value__[0] == 0:
            return "-"
        else:
            return "+"

    # * Sets the specified Byte to a Value
    def set_byte(self, byte, val):
        byte_overflow = (byte > self.__size__) or (byte < 1)
        if byte_overflow:
            raise Exception("Overflow: byte number has to be in range from 1 to Word.size")
        
        value = val % BYTE_SIZE
        self.__value__[byte] = value
    
    @sign.setter
    def sign(self, sign):
        if sign == 0:
            self.__value__[0] = 0
        else:
            self.__value__[0] = 1

    def __repr__(self) -> str:
        if self.__value__[0] == 0:
            repr = "- "
        else:
            repr = "+ "
        
        for i in range(1, self.__size__ + 1):
            repr += str(self.__value__[i]) + " "

        return repr
    
    def get_field(self, L, R):
        R_field_overflow = (R < 0) or (R > self.__size__)
        L_field_overflow = (L < 0) or (L > R)
        field_overflow = L_field_overflow or R_field_overflow
        if field_overflow:
            raise Exception("Field overflow: the field must satisfy the conditions: 0 <= L <= R <= Word.size")
        
        return self.__value__[L: R+1]


class Word(GenericWord):
    def __init__(self) -> None:
        super().__init__(5)


class AddressWord(GenericWord):
    def __init__(self) -> None:
        super().__init__(2)

    @property
    def address(self):
        return self.__value__[1]*BYTE_SIZE + self.__value__[2]


class Command(Word):
    def __init__(self, name) -> None:
        super().__init__()
        self.__name__ = name

    def __repr__(self) -> str:
        repr = self.__name__ + ": "
        if self.__value__[0] == 0:
            repr += "- "
        else:
            repr += "+ "
        
        for i in range(1, self.__size__ + 1):
            repr += str(self.__value__[i]) + " "

        return repr

    @property
    def name(self):
        return self.__name__

    @property
    def code(self):
        return self.__value__[6]
    
    @property
    def field(self):
        return self.__value__[4]
    
    @property
    def index(self):
        return self.__value__[3]
    
    @property
    def address(self):
        abs_address = self.__value__[1]*BYTE_SIZE + self.__value__[2]
        if self.sign == "-":
            factor = -1
        else:
            factor = 1
        return factor * abs_address

### The machine core

In [34]:
class MIX:
    def __init__(self):
        self.__rA__ = Word()
        self.__rX__ = Word()
        self.__rI1__ = AddressWord()
        self.__rI2__ = AddressWord()
        self.__rI3__ = AddressWord()
        self.__rI4__ = AddressWord()
        self.__rI5__ = AddressWord()
        self.__rI6__ = AddressWord()
        self.__rJ__ = AddressWord()
        self.__overflow_trigger__ = False # or True
        self.__comparison_flag__ = LESS # or EQUAL or GREATER
        
        self.__memory__ = np.ndarray((RAM_SIZE), dtype=object)
        for i in range(RAM_SIZE):
            self.__memory__[i] = Word()

        # TODO Input-Output devices

    @property
    def rA(self):
        return self.__rA__

    @property
    def rX(self):
        return self.__rX__

    @property
    def rI1(self):
        return self.__rI1__

    @property
    def rI2(self):
        return self.__rI2__

    @property
    def rI3(self):
        return self.__rI3__
    
    @property
    def rI4(self):
        return self.__rI4__

    @property
    def rI5(self):
        return self.__rI5__

    @property
    def rI6(self):
        return self.__rI6__

    @property
    def rJ(self):
        return self.__rJ__

    @property
    def overflow(self):
        return self.__overflow_trigger__

    @property
    def cmp_flag(self):
        return self.__comparison_flag__
    
    
    # * commands
    def command(self, cmd):
        C = cmd.code
        F = cmd.field
        I = cmd.index
        A = cmd.address        


In [19]:
c = Command('LDA')
c

LDA: + 0 0 0 0 0 

In [20]:
c.address

0

In [21]:
I = AddressWord()
I

+ 0 0 

In [22]:
I.address

0

In [33]:
a = np.ndarray((5), dtype=object)
for i in range(5):
    a[i] = Word()

a

array([+ 0 0 0 0 0 , + 0 0 0 0 0 , + 0 0 0 0 0 , + 0 0 0 0 0 ,
       + 0 0 0 0 0 ], dtype=object)